## Scraping part

## Firstly, we need to install Scrapy library. Let's do it with pip 3.11 version

In [1]:
!pip3.11 install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.4 MB/s eta 0:00:00


In [2]:
import os

# Creates a directory if it doesn’t already exist and navigates into it.
project_name = "my_scrapy_project"
os.makedirs(project_name, exist_ok=True)
os.chdir(project_name)

## Secondly, we need to create and start the project


In [3]:
!scrapy startproject wikipedia

New Scrapy project 'wikipedia', using template directory '/usr/local/lib/python3.11/dist-packages/scrapy/templates/project', created in:
    /content/my_scrapy_project/wikipedia

You can start your first spider with:
    cd wikipedia
    scrapy genspider example example.com


## Then we need to create a spider

In [4]:
os.chdir("wikipedia")
!scrapy genspider films_spider en.wikipedia.org

Created spider 'films_spider' using template 'basic' in module:
  wikipedia.spiders.films_spider


## Class of spider with start URL, domain and all scraping functions

In [10]:
%%writefile /content/my_scrapy_project/wikipedia/wikipedia/spiders/films_spider.py

import scrapy
import re

# Spider class
class HighestGrossingFilmsSpider(scrapy.Spider):
    name = 'films_spider'
    allowed_domains = ['en.wikipedia.org']
    start_urls = ['https://en.wikipedia.org/wiki/List_of_highest-grossing_films']

    # Extracts the talbe with the films
    def parse(self, response):

        table = response.xpath('//table[contains(@class, "wikitable") and contains(@class, "sortable")]')

        # Shows error message if table wasn't foundf
        if not table:
            self.logger.error("Table not found...")
            return

        # Extracting film data
        for row in table.xpath('.//tr')[1:]:
            rank = row.xpath('.//td[1]//text()').get()
            title = row.xpath('.//th//i//a//text()').get()
            worldwide_gross = row.xpath('.//td[3]//text()').getall()
            year = row.xpath('.//td[4]//text()').get()
            film_url = row.xpath('.//th//i//a/@href').get()

            # Cleaning the extracted data
            if worldwide_gross:

                worldwide_gross = ''.join(worldwide_gross).strip()

                dollar_index = worldwide_gross.find('$')

                if dollar_index != -1:
                    worldwide_gross = worldwide_gross[dollar_index:]
                else:
                    worldwide_gross = ''

            if rank:
                rank = rank.strip()
            if title:
                title = title.strip()
            if year:
                year = year.strip()

            # Opens film pages
            if film_url:
                film_url = response.urljoin(film_url)
                yield scrapy.Request(
                    url=film_url,
                    callback=self.parse_film_page,
                    meta={
                        'rank': rank,
                        'title': title,
                        'worldwide_gross': worldwide_gross,
                        'year': year,
                    }
                )
            else:

                yield {
                    'rank': rank,
                    'title': title,
                    'worldwide_gross': worldwide_gross,
                    'year': year,
                    'director': None,
                    'country': None,
                }

    # Extracting film director and country
    def parse_film_page(self, response):

        director = response.xpath('//th[contains(text(), "Directed by")]/following-sibling::td//text()').getall()

        # Cleaning director data
        director = [
            name.strip() for name in director
            if name.strip()
            and not name.startswith(".mw-parser-output")
            and not name.startswith("[")
            and not name.endswith("]")
        ]

        director = ''.join(director).strip()

        director = re.sub(r'\d+', '', director)

        director = re.sub(r'([a-z])([A-Z])', r'\1, \2', director)

        director = ', '.join([name.strip() for name in director.split(',') if name.strip()])

        country = response.xpath('//th[contains(text(), "Country") or contains(text(), "Countries")]/following-sibling::td//text()').getall()

        if not country:
            country = response.xpath('//div[contains(@class, "infobox")]//th[contains(text(), "Country") or contains(text(), "Countries")]/following-sibling::td//text()').getall()

        # Cleaning country data
        country = [
            name.strip() for name in country
            if name.strip()
            and not name.startswith(".mw-parser-output")
            and not name.startswith("[")
            and not name.endswith("]")
        ]

        country = ''.join(country).strip()

        country = re.sub(r'\d+', '', country)

        country = re.sub(r'([a-z])([A-Z])', r'\1, \2', country)

        country = ', '.join([name.strip() for name in country.split(',') if name.strip()])

        # Collects data for output
        yield {
            'rank': response.meta['rank'],
            'title': response.meta['title'],
            'worldwide_gross': response.meta['worldwide_gross'],
            'year': response.meta['year'],
            'director': director if director else None,
            'country': country if country else None,
        }

Overwriting /content/my_scrapy_project/wikipedia/wikipedia/spiders/films_spider.py


## Overwriting data in output.json in case it's not empty


In [20]:
%%writefile output.json

Overwriting output.json


## Scraping data from the website to the JSON

In [21]:
!scrapy crawl films_spider -o output.json

2025-03-02 15:13:35 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: wikipedia)
2025-03-02 15:13:35 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-03-02 15:13:35 [scrapy.addons] INFO: Enabled addons:
[]
2025-03-02 15:13:35 [asyncio] DEBUG: Using selector: EpollSelector
2025-03-02 15:13:35 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-03-02 15:13:35 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-03-02 15:13:35 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-03-02 15:13:35 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-03-0

## DataBase part

## Creating database using SQLite and fullfiling it with the scraped data from the JSON

In [26]:
%%writefile films.db

Overwriting films.db


In [27]:
import sqlite3
import json

# Connect to (or create) the SQLite database file
conn = sqlite3.connect('films.db')

# Initialize cursor object to work with database
cursor = conn.cursor()

# Creates films database
cursor.execute('''
CREATE TABLE IF NOT EXISTS films (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_year INTEGER,
    director TEXT,
    box_office TEXT,
    country TEXT
)
''')

# Opens json file with scraped data
with open('output.json', 'r', encoding='utf-8') as file:
    films_data = json.load(file)

# Inserts films data into the database
for film in films_data:

    box_office = film['worldwide_gross'].replace('$', '')

    cursor.execute('''
    INSERT INTO films (title, release_year, director, box_office, country)
    VALUES (?, ?, ?, ?, ?)
    ''', (film['title'], int(film['year']), film['director'], box_office, film['country']))

# Saves data to the database
conn.commit()

# Closes the connection to the database
conn.close()

## Extracting all the data from DataBase to JSON file

In [30]:
import json

# Reconnect to the database file
conn = sqlite3.connect('films.db')

# Initialize cursor object to work with database
cursor = conn.cursor()

# Retrieves all data from the database
cursor.execute('SELECT * FROM films')
rows = cursor.fetchall()

# Converting a data from the databse to the JSON
films_list = []
for row in rows:
    film = {
        "id": row[0],
        "title": row[1],
        "release_year": row[2],
        "director": row[3],
        "box_office": row[4],
        "country": row[5]
    }
    films_list.append(film)

# Closing the database connection
conn.close()

# Writing data to the JSON file
with open('films_exported.json', 'w', encoding='utf-8') as json_file:
    json.dump(films_list, json_file, ensure_ascii=False, indent=4)